In [ ]:
import os
import sys
curruser = os.environ.get('USER')

_labdata = os.environ.get("LABDATA_PYSPARK")
sys.path.insert(0, _labdata)

if curruser in os.listdir("/opt/workspace/"):
    sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
    sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
#     sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/'.format(user=curruser))
else:
    sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser))
    sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    # sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime as dt

from transliterate import translit

from lib.spark_connector import SparkConnector
from lib.sparkdb_loader import *
from lib.connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from collections import ChainMap

from lib.config import *
from lib.tools import *

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))

def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

    return _typed_udf_wrapper

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


In [ ]:
def map_product_group(p_crm_prod, b_rdd, cols):

    import pandas as pd

    if (p_crm_prod is not None) and (p_crm_prod not in ('', ' ')):
        v_crm_prod = p_crm_prod.upper()
        if v_crm_prod in ('ЭКСПРЕСС ОВЕРДРАФТ','ОВЕРДРАФТНОЕ КРЕДИТОВАНИЕ',
                          'ОБОРОТНОЕ КРЕДИТОВАНИЕ', 'КРЕДИТ'):
            v_crm_prod =  'КРЕДИТОВАНИЕ'
        elif v_crm_prod in ('ДИСТАНЦИОННОЕ БАНКОВСКОЕ ОБСЛУЖИВАНИЕ'):
            v_crm_prod =  'ДИСТАНЦИОННОЕ ОБСЛУЖИВАНИЕ';
        elif v_crm_prod in ('ОТКРЫТИЕ СЧЕТА В ВАЛЮТЕ','ОТКРЫТИЕ СЧЕТА В РУБЛЯХ',
                            'СПЕЦСЧЕТА','РКО','РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ'):
            v_crm_prod = 'РКО'
        elif v_crm_prod in ('ПРОДУКТЫ ИНКАССАЦИИ'):
            v_crm_prod =  'ИНКАССАЦИЯ'
        elif v_crm_prod in ('НЕСНИЖАЕМЫЙ ОСТАТОК'):
            v_crm_prod = 'ПРОДУКТЫ ПРИВЛЕЧЕНИЯ СРЕДСТВ';
        elif 'ПАКЕТ УСЛУГ' in v_crm_prod:
            v_crm_prod = 'ПАКЕТЫ УСЛУГ'
        elif v_crm_prod in ('ВЭД И ВАЛЮТНЫЙ КОНТРОЛЬ'):
            v_crm_prod = 'ВАЛЮТНЫЙ КОНТРОЛЬ'
        elif v_crm_prod in ('АККРЕДИТИВЫ'):
            v_crm_prod = 'ДОКУМЕНТАРНЫЕ ОПЕРАЦИИ'
        elif v_crm_prod in ('КОРПОРАТИВНАЯ КАРТА'):
            v_crm_prod = 'БИЗНЕС-КАРТА'
        elif v_crm_prod in ('БИЗНЕС-КАРТА'):
            v_crm_prod = 'БИЗНЕС-КАРТА'
        elif v_crm_prod in ('ДЕПОЗИТЫ (СРОЧНЫЕ)'):
            v_crm_prod = 'ПРОДУКТЫ ПРИВЛЕЧЕНИЯ СРЕДСТВ'
        elif v_crm_prod in ('АКТИВИЗАЦИЯ РКО'):
            v_crm_prod = None

        dct = {col: [b_rdd.value[i][col] for i in range(len(b_rdd.value))] for col in cols}
        df = pd.DataFrame.from_dict(dct, orient='columns')
        if v_crm_prod is not None:
            try:
                v_prod_group_trg = df.loc[df['P_NAME'].str.contains(v_crm_prod),'P_GROUP'].tolist()[0]
                return v_prod_group_trg.lower()
            except:
                v_prod_group_trg = v_crm_prod
                return v_prod_group_trg.lower()
        else:
            return None
    else:
        return None

def udf_map_product_group(b_rdd, cols):
    return f.udf(lambda p: map_product_group(p, b_rdd, cols))


In [ ]:
CONN_SCHEMA='sbx_team_digitcamp'

In [ ]:
%%time
sp = spark(schema=CONN_SCHEMA,
           dynamic_alloc=False,
           numofinstances=10,
           numofcores=8,
           executor_memory='40g',
           driver_memory='40g',
           kerberos_auth=True,
           process_label="SAS_ANALYT_"
           )

hive = sp.sql
print(sp.sc.version)

In [ ]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('', '', ld._get_dsn('')) 
db.connect()
curs = db.cursor

In [ ]:
table_name = 'mw_atb_segmen_prod_group'
query = '''
select /*+ parallel (12) */
          *
FROM {}
--where rownum < 1000
'''.format(table_name)

df = pd.read_sql(query, con=db.connection)

In [ ]:
df.loc[df['P_NAME'].str.contains('ОВЕРДР'),['P_NAME', 'P_GROUP']]

In [ ]:
set(df['P_GROUP'].values.tolist())

In [ ]:
len(set(df['P_GROUP'].values.tolist()))

In [ ]:
mw_prod_group = hive.sql('''select * from {}.mw_atb_segmen_prod_group'''.format(CONN_SCHEMA))
df = mw_prod_group.take(sdf.count())
b_rdd = sp.sc.broadcast(df)
cols = mw_prod_group.columns

In [ ]:
mw_prod_group.show()

In [ ]:
mw_prod_group.filter("lower(P_NAME) like '%договор%'").collect()

In [ ]:
# conn_schema = 'sbx_team_digitcamp'
table_name = 'MA_CMDM_MA_AGREEMENT' #'MA_CMDM_MA_DEAL_NEW'

In [ ]:
agrmnt = hive.sql('''select distinct FIN_AGRMNT_TYPE_DESC from sbx_team_digitcamp.MA_CMDM_MA_AGREEMENT''').collect()

In [ ]:
fin_agrmnt_type = [col['FIN_AGRMNT_TYPE_DESC'] for col in agrmnt]
fin_agrmnt_type

## Analyse U_BASE_DEAL

In [ ]:
u_base_deals = hive.table("cib_custom_cib_ml360.u_base_deals")

In [ ]:
udeals_innprod_uniq = u_base_deals.select("inn", "product_id").distinct().collect()
udeal_dct = [row.asDict() for row in udeals_innprod_uniq]
udeal_df = pd.DataFrame.from_dict(udeal_dct)

In [ ]:
udeal_df.head()

In [ ]:
gr_cols = \
["inn", 
 "product_id"
]

udeals_gr = \
u_base_deals.groupBy(gr_cols).agg(f.countDistinct("product_id").alias('cnt'))\
                    .select(gr_cols+['cnt']).orderBy(f.col('cnt').desc()).toPandas()

In [ ]:
udeals_gr.sort_index(ascending=False)

## MA_CMDM_MA_DEAL_NEW

In [ ]:
conn_schema = 'sbx_team_digitcamp'
table_name = 'MA_CMDM_MA_DEAL_NEW'

startdt = dt.now()

#/////////////////////////////////////////////////////////////////////////////////////
descr = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
hasPartitioned = len([item.asDict() for item in descr if item.asDict()['col_name'] =='# Partition Information']) > 0
if hasPartitioned:
    try:
        parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
        max_part = sorted(parts,reverse=True)[0]['partition']
        extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
        ext = extract_date.search(max_part)
        try:
            max_trunc_dt = ext.group(0)
        except:
            max_trunc_dt = None
    except (AnalysisException, IndexError):
        max_trunc_dt = None
else:
    max_trunc_dt = None
#/////////////////////////////////////////////////////////////////////////////////////

if max_trunc_dt is not None:
  sql = '''select max(create_dt) from {}.{} where creation_month = '{}' '''.format(conn_schema,table_name,max_trunc_dt)
else:
  sql = '''select max(create_dt) from {}.{}'''.format(conn_schema,table_name)

max_dt = hive.sql(sql).collect()
max_dt = max_dt[0]['max(create_dt)']
max_resp_dt_str = dt.strftime(max_dt, format='%Y-%m-%d %H:%M:%S.%f')

enddt = dt.now() - startdt

_str = max_resp_dt_str #json.dumps(_dct, ensure_ascii=False, indent=3)

print("total calculation time: {}".format(enddt.total_seconds()))
print("max(create_dt): {}".format(_str))

In [ ]:
query = \
'''
select 
        distinct
        --inn, 
        --create_dt,
        --complete_dt,
        host_prod_id
        --close_reason_type_name,
        --appl_stage_name,
        --appl_type_name,
        --channel_mmb_ms,
        --channel_mmb_ckr,
        --channel_mmb_mkk,
        --channel_dzo
        --creation_month
from sbx_team_digitcamp.MA_CMDM_MA_DEAL_NEW
'''

In [ ]:
%%time
macmdm_deal_prod_uniq = hive.sql(query).toPandas()
# macmdm_deal_innprod_uniq = macmdm_deal.select("inn", "host_prod_id").distinct().collect()

In [ ]:
sp.sc.stop()

In [ ]:
macmdm_deal_prod_uniq = macmdm_deal_prod_uniq.fillna('-1')

In [ ]:
macmdm_deal_prod_uniq.host_prod_id.value_counts()

In [ ]:
prod_cnt = \
macmdm_deal_prod_uniq.groupby('host_prod_id')\
                        .agg(cnt=('host_prod_id', 'count')).sort_values('cnt', ascending=False)

In [ ]:
prod_cnt.head(20)

In [ ]:
# madeal_dct = [row.asDict() for row in macmdm_deal_innprod_uniq]
# madeal_df = pd.DataFrame.from_dict(madeal_dct)

In [ ]:
# madeal_df.rename(columns={'host_prod_id': 'product_id'}, inplace=True)

In [ ]:
# madeal_df.describe()

In [ ]:
udeal_df.describe()

In [ ]:
madeal_df['label'] = 'ma_deal'
out = udeal_df.merge(madeal_df, on=['inn', 'product_id'], suffixes=('_madeal', '_udeal'), how='outer')

In [ ]:
out = out.drop_duplicates()

In [ ]:
out.info()

In [ ]:
sum(out.label.isnull())

In [ ]:
out.describe()

In [ ]:
len(set(madeal_df.inn.values) - set(udeal_df.inn.values))

In [ ]:
len(set(madeal_df.product_id.values) - set(udeal_df.product_id.values))

## Join Deals with Agreements and Prod Dict

In [ ]:
sql = \
'''
with deals as (
select 
        inn, 
        create_dt,
        complete_dt,
        host_prod_id,
        close_reason_type_name,
        appl_stage_name,
        --appl_type_name,
        --channel_mmb_ms,
        --channel_mmb_ckr,
        --channel_mmb_mkk,
        --channel_dzo
        creation_month
from sbx_team_digitcamp.MA_CMDM_MA_DEAL_NEW
where (create_dt is not Null) and
      (appl_stage_name like '%Закрыта/Заключена%') and
      (close_reason_type_name is Null)
),

prods as (
select 
       CRM_PRODUCT_ID, 
       PRODUCT_CRM_NM, 
       PRODUCT_CD_MMB,
       PRODUCT_FULL_NM,
       PRODUCT_SUBGROUP,
       PRODUCT_GROUP
       --CRM_ACTIVE_FLG
from sbx_team_digitcamp.ma_dict_v_product_dict
),

deals_prods as (
select 
    deals.inn,
    deals.create_dt,
    deals.complete_dt,
    deals.appl_stage_name,
    prods.PRODUCT_CRM_NM,
    prods.CRM_PRODUCT_ID,
    prods.PRODUCT_SUBGROUP
from deals
left join prods on deals.host_prod_id = prods.CRM_PRODUCT_ID
where 
(prods.PRODUCT_SUBGROUP = "Бизнес-карта") and 
(timestamp(deals.creation_month) >= timestamp('2021-01-01'))
),

agrmnts as(
select 
    AGREEMENT_ID,
    INN,
    ACCT_SIGNED_DT,
    ACCT_CLOSE_DT,
    FIN_AGRMNT_TYPE_DESC
from sbx_team_digitcamp.MA_CMDM_MA_AGREEMENT
where (FIN_AGRMNT_TYPE_DESC = 'Бизнес-карта') and
      (acct_signed_str is not Null) and
      (timestamp(acct_signed_str) between timestamp('2021-01-01') and timestamp('2022-01-01'))
)

select 
    dls.inn,
    dls.create_dt,
    dls.complete_dt,
    agrmnts.ACCT_SIGNED_DT,
    agrmnts.ACCT_CLOSE_DT,
    agrmnts.AGREEMENT_ID,
    dls.appl_stage_name,
    dls.PRODUCT_CRM_NM,
    dls.CRM_PRODUCT_ID,
    dls.PRODUCT_SUBGROUP 
from deals_prods dls
left join agrmnts on (dls.inn = agrmnts.INN) and
                     (abs(unix_timestamp(agrmnts.ACCT_SIGNED_DT) - 
                          unix_timestamp(dls.create_dt)) <= 30 * 24 * 60 * 60
                     ) and
                     (agrmnts.ACCT_SIGNED_DT < dls.create_dt)
where agrmnts.ACCT_SIGNED_DT is not Null
'''

In [ ]:
hive.sql(sql).show(20)

In [ ]:
res = hive.sql(sql)

In [ ]:
res.drop_duplicates().select('inn').distinct().count()

In [ ]:
prod_dict = hive.sql('''select CRM_PRODUCT_ID, 
                               PRODUCT_CRM_NM, 
                               PRODUCT_CD_MMB,
                               PRODUCT_FULL_NM,
                               PRODUCT_SUBGROUP,
                               PRODUCT_GROUP,
                               CRM_ACTIVE_FLG
                        from sbx_team_digitcamp.ma_dict_v_product_dict
                     ''')

In [ ]:
prod_dict.filter("lower(PRODUCT_SUBGROUP) like '%бизнес%'").show(10)

In [ ]:
parts = hive.sql("show partitions {}.{}".format(CONN_SCHEMA,table_name)).collect()
parts

In [ ]:
hive.sql('''select * from {}.{} where (inn = '2221191879') and 
                                      (acct_signed_str between '2021-02-01' and '2021-04-01')'''.format(CONN_SCHEMA, table_name)).show(40)

In [ ]:
deals = \
hive.sql(''' select 
                    inn, 
                    create_dt,
                    complete_dt,
                    host_prod_id,
                    --close_reason_type_name,
                    appl_stage_name
                    --appl_type_name,
                    --channel_mmb_ms,
                    --channel_mmb_ckr,
                    --channel_mmb_mkk,
                    --channel_dzo  
             from {}.{} where (inn = '2221191879') and
                              (creation_month between '2021-02-01' and '2021-04-01') '''.format(CONN_SCHEMA, "MA_CMDM_MA_DEAL_NEW"))

In [ ]:
conditions = (prod_dict.CRM_PRODUCT_ID == deals.host_prod_id)
res = deals.join(prod_dict, on=conditions)

In [ ]:
res.show(20)

In [ ]:
deals = \
hive.sql(''' select 
                    inn, 
                    create_dt,
                    complete_dt,
                    host_prod_id,
                    --close_reason_type_name,
                    appl_stage_name
                    --appl_type_name,
                    --channel_mmb_ms,
                    --channel_mmb_ckr,
                    --channel_mmb_mkk,
                    --channel_dzo  
             from {}.{} where (inn = '2221191879') and
                              (creation_month in ('2021-02-01', 
                                                  '2021-03-01',
                                                  '2021-04-01',
                                                  '2021-05-01') )'''.format(CONN_SCHEMA, "MA_CMDM_MA_DEAL_NEW"))

In [ ]:
conditions = (prod_dict.CRM_PRODUCT_ID == deals.host_prod_id)
res = deals.join(prod_dict, on=conditions)

In [ ]:
res.show()

In [ ]:
startdt = dt.now()

#/////////////////////////////////////////////////////////////////////////////////////
descr = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
hasPartitioned = len([item.asDict() for item in descr if item.asDict()['col_name'] =='# Partition Information']) > 0
if hasPartitioned:
    try:
        parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
        max_part = sorted(parts, reverse=True)[0]['partition']
        extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
        ext = extract_date.search(max_part)
        try:
            max_trunc_dt = ext.group(0)
        except:
            max_trunc_dt = None
    except (AnalysisException, IndexError):
        max_trunc_dt = None
else:
    max_trunc_dt = None
#/////////////////////////////////////////////////////////////////////////////////////


In [ ]:
if max_trunc_dt is not None:
  sql = '''select * from {}.{} where creation_month in 
                 ('2021-06-01', '2021-05-01',' 2021-04-01') '''.format(conn_schema, table_name)
else:
  sql = '''select * from {}.{}'''.format(conn_schema,table_name)

deals = hive.sql(sql)
# max_resp_dt_str = dt.strftime(max_dt, format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
deals.select('appl_stage_name').distinct().show(20)

## Cancelled Deals

In [ ]:
deals_d = \
deals.filter((f.col('appl_stage_name').like("%Закрыта/Отказ%"))&
             (~f.isnull(f.col('close_reason_type_name')))&
             (~f.isnull(f.col('product_cd_mmb')))
            ) 
    
deals_d = deals_d.select('inn', 'product_cd_mmb', f.lit(1).alias('flg_deal_deny'))

deals_deny = deals_d.groupBy('product_cd_mmb').agg(f.count('flg_deal_deny').alias('cnt_deals_deny')).orderBy(f.col('cnt_deals_deny').desc())

In [ ]:
deals_deny.cache()

In [ ]:
deals_deny.show()

## Succeeded Deals

In [ ]:
deals_s = \
deals.filter(f.col('appl_stage_name').like("%Закрыта/Заключена%")) \
     .filter(f.isnull(f.col('close_reason_type_name'))&(~f.isnull('product_cd_mmb'))
            ) \
     .select('inn', 
             'product_cd_mmb', 
             f.col('SUM_RUB').cast(IntegerType())
            ).orderBy(f.col('SUM_RUB').desc())

deals_succ = deals_s.groupBy('product_cd_mmb').agg(f.count('SUM_RUB').alias('cnt_deals_succ'),
                                                   f.sum('SUM_RUB').alias('total_sum_rub')
                                                  ).orderBy(f.col('cnt_deals_succ').desc())

In [ ]:
deals_succ.cache()

In [ ]:
deals_succ.show()

## MA_PRODUCT_DICT

In [ ]:
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(conn_schema, 'ma_dict_v_product_dict'))
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null) and (PRODUCT_SHORT_NM is not Null)")

prod_dct.columns

## MA_MMB_OFFER_NONTOP

In [ ]:
nontop = hive.sql("select * from {}.{} where (INN is not Null) and (CREATE_DTTM >= timestamp('2021-04-01'))".format(conn_schema, 'MA_MMB_OFFER_NONTOP'))

In [ ]:
def getProdCDFromId(product_dict_nm):
    def get_product(product_id, product_dict_nm):
        if product_id is None:
            return None

        for _id in product_dict_nm:
            if product_id == _id:
                return product_dict_nm[_id]

        return list([])

    return f.udf(lambda x: get_product(x,product_dict_nm), ArrayType(StringType()))

w2=Window.partitionBy('HASHINN', 'CRM_PRODUCT_ID').orderBy("COEF_PRODUCT_CD")

product_dict_sp = prod_dct.select('ID', 
                                  'CRM_PRODUCT_ID', 
                                  'PRODUCT_CD_MMB',
                                  'PRODUCT_SHORT_NM',
                                  'PRODUCT_GROUP', 
                                  'PRODUCT_SUBGROUP')

product_dict_pd = product_dict_sp.collect()
product_dict_nm = {row.ID: (row.CRM_PRODUCT_ID, 
                            row.PRODUCT_CD_MMB, 
                            row.PRODUCT_SHORT_NM,
                            row.PRODUCT_GROUP,
                            row.PRODUCT_SUBGROUP
                           ) for row in product_dict_pd}

nontop = nontop.withColumn('PROD_LST', getProdCDFromId(product_dict_nm)('PRODUCT_ID'))

nontop = nontop.select(['INN', f.col('COEF_PRODUCT_CD').alias("POTENTIAL")] +
                       [f.col('PROD_LST')[0].alias('CRM_PRODUCT_ID'), 
                        f.col('PROD_LST')[1].alias('PRODUCT_CD_MMB'),
                        f.col('PROD_LST')[2].alias('PRODUCT_SHORT_NM'),
                        f.col('PROD_LST')[3].alias('PRODUCT_GROUP'),
                        f.col('PROD_LST')[4].alias('PRODUCT_SUBGROUP')
                       ]).filter("PRODUCT_CD_MMB is not Null")

In [ ]:
nontop.show()

In [ ]:
mean_nontop = nontop.groupBy('CRM_PRODUCT_ID').agg(f.mean('POTENTIAL').cast(IntegerType()).alias('MEAN_POTENTIAL'),
                                                   f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                   f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                   f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                   f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP')
                                                  ).orderBy(
                                                            f.col("MEAN_POTENTIAL").desc()
                                                           )

mean_nontop.cache()

In [ ]:
mean_nontop.count()

In [ ]:
mean_nontop.show()

## OFFER_PRIORITY 

In [ ]:
priority = hive.sql("select * from {}.{} where inn is not Null and load_dttm >= timestamp('2021-04-01')".format(conn_schema, 'OFFER_PRIORITY'))

In [ ]:
priority_part = priority.select('INN', 
                                'CRM_PRODUCT_ID', 
                                f.col('offer_priority').alias("POTENTIAL") 
                                )

In [ ]:
priority_part = priority_part.join(product_dict_sp, on='CRM_PRODUCT_ID', how='left_outer').select('INN', 
                                                                                                  'CRM_PRODUCT_ID', 
                                                                                                  'PRODUCT_CD_MMB',
                                                                                                  'PRODUCT_SHORT_NM',
                                                                                                  'PRODUCT_GROUP',
                                                                                                  'PRODUCT_SUBGROUP',
                                                                                                  'POTENTIAL').filter("PRODUCT_CD_MMB is not Null")

In [ ]:
mean_priority = priority_part.groupBy('CRM_PRODUCT_ID').agg(f.mean('POTENTIAL').cast(IntegerType()).alias('MEAN_POTENTIAL'),
                                                            f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                            f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                            f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                            f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP')
                                                           ).orderBy(f.col("MEAN_POTENTIAL").desc())

mean_priority.cache()

In [ ]:
mean_priority.cache()

In [ ]:
mean_priority.count()

## Union Offers

In [ ]:
union_offers = mean_priority.unionAll(mean_nontop).select('CRM_PRODUCT_ID', 
                                                          'PRODUCT_CD_MMB', 
                                                          'PRODUCT_SHORT_NM',
                                                          'PRODUCT_GROUP',
                                                          'PRODUCT_SUBGROUP',
                                                          'MEAN_POTENTIAL')
union_offers = union_offers.drop_duplicates()

In [ ]:
union_offers = union_offers.join(deals_succ, on=['PRODUCT_CD_MMB'], how='left_outer').select(
                                                                                             'CRM_PRODUCT_ID',
                                                                                             'PRODUCT_CD_MMB',
                                                                                             'PRODUCT_SHORT_NM',
                                                                                             'PRODUCT_GROUP',
                                                                                             'PRODUCT_SUBGROUP',    
                                                                                             'MEAN_POTENTIAL',
                                                                                             'cnt_deals_succ',
                                                                                             'total_sum_rub'          
                                                                                             )


union_offers = union_offers.join(deals_deny, on=['PRODUCT_CD_MMB'], how='left_outer').select( 
                                                                                             'CRM_PRODUCT_ID',
                                                                                             'PRODUCT_CD_MMB',
                                                                                             'PRODUCT_SHORT_NM',
                                                                                             'PRODUCT_GROUP',
                                                                                             'PRODUCT_SUBGROUP',    
                                                                                             'MEAN_POTENTIAL',
                                                                                             'cnt_deals_deny',
                                                                                             'cnt_deals_succ',
                                                                                             'total_sum_rub'
                                                                                             )

union_offers = union_offers.drop_duplicates()

In [ ]:
fin = union_offers.groupBy('CRM_PRODUCT_ID').agg( f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                  f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                  f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                  f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP'),
                                                  f.first('MEAN_POTENTIAL').alias('MEAN_POTENTIAL'),
                                                  f.sum('cnt_deals_succ').alias('cnt_deals_succ'.upper()),
                                                  f.sum('cnt_deals_deny').alias('cnt_deals_deny'.upper()),
                                                  f.sum('total_sum_rub').alias('total_sum_rub'.upper()),
                                                  ).orderBy(f.col("MEAN_POTENTIAL").desc(),
                                                            f.col("cnt_deals_succ").desc()
                                                           )

In [ ]:
df = fin.filter(~f.isnull('cnt_deals_succ')).toPandas()

In [ ]:
df.head()

In [ ]:
df.to_excel("/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/site_idprod_deals_rank.xlsx", index = False, encoding='utf8')